In [1]:
!pip install pymorphy2 stop-words annoy

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
from tqdm.notebook import tqdm


import gensim.models
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from gensim.models.phrases import Phrases, Phraser

In [3]:
from collections import defaultdict

In [4]:
from google.colab import files

uploaded = files.upload()

In [4]:
df = pd.read_excel('/content/отзывы за лето.xls')

In [5]:
df.head(3)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [7]:
stop_words = set(stopwords.words('russian'))
stop_words.add('это')
morph = MorphAnalyzer()

In [8]:
def preprocess_text(text):
  text = str(text)
  text = re.sub('@[\w]*', '', text)
  text = text.lower()
  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'[^а-яА-ЯЁё]', ' ', text)
  text = " ".join([w for w in text.split() if (len(w) > 3) and len(w) < 11])
  text = [w for w in text.split() if w not in stop_words]
  text = " ".join([morph.parse(w)[0].normal_form for w in text])
  if len(text) == 0:
    text = None
  return text

In [9]:
df['text'] = df['Content'].apply(preprocess_text)

In [10]:
df.dropna(axis=0, how='any', inplace=True)

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df.head(3)

,Rating,Content,Date,text
0,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
1,5,Отлично все,2017-08-14,отлично
2,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать работа антивирус далёкий никуда...


In [13]:
df.shape

(19310, 4)

In [14]:
sent = [row.split() for row in df['text']]

In [15]:
sent[0]

['целое',
 'удобноной',
 'приложение',
 'минус',
 'хотеть',
 'слишком',
 'большой',
 'доступ',
 'данные',
 'телефон',
 'приходиться',
 'режим']

In [16]:
phrases = Phrases(sent, min_count=10, progress_per=100)

In [17]:
bigram = Phraser(phrases)

In [18]:
sentences = bigram[sent]

In [19]:
len(sentences)

19310

Ищем самые частые слова

In [20]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

5951

In [21]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['приложение',
 'очень',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'спасибо',
 'нравиться',
 'хороший',
 'отличный']

In [22]:
from gensim.models import Word2Vec

In [23]:
w2v_model = Word2Vec(vector_size=300, window=5, min_count=10)

In [24]:
w2v_model.build_vocab(sentences)

In [25]:
vocab_len = len(w2v_model.wv)

In [26]:
vocab_len

1065

In [27]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(300)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in w2v_model.wv:
            vector_w2v += w2v_model.wv[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [28]:
arr_vect = []
for txt in tqdm(df['text']):
    arr_vect.append(get_vect_mean(txt))

  0%|          | 0/19310 [00:00<?, ?it/s]

In [29]:
len(arr_vect)

19310

In [30]:
X = np.asarray(arr_vect)

In [31]:
X.shape

(19310, 300)

In [32]:
df.head(3)

,Rating,Content,Date,text
0,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
1,5,Отлично все,2017-08-14,отлично
2,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать работа антивирус далёкий никуда...


Модель

In [33]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [34]:
from tensorflow.keras import backend as K

In [35]:
def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
import tensorflow as tf

In [38]:
num_classes = len(df.Rating.unique())

In [39]:
le = LabelEncoder()
enc_labels = le.fit_transform(df['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [40]:
Y = tf.keras.utils.to_categorical(enc_labels, num_classes=num_classes)

In [41]:
Y.shape

(19310, 5)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [43]:
X_train.shape

(12937, 300)

In [44]:
y_train.shape

(12937, 5)

In [45]:
model = Sequential([
    Embedding(input_dim=vocab_len, output_dim=128, input_length=300),
    Conv1D(128, 3),
    GlobalMaxPool1D(),
    Dense(10),
    Activation("relu"),
    Dense(num_classes),
    Activation('softmax')
    ])


In [46]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[custom_f1])

In [47]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
46/46 [==============================] - 34s 701ms/step - loss: 1.1098 - custom_f1: 0.5461 - val_loss: 1.0235 - val_custom_f1: 0.7117
Epoch 2/20
46/46 [==============================] - 32s 687ms/step - loss: 1.0180 - custom_f1: 0.6959 - val_loss: 1.0142 - val_custom_f1: 0.7117
Epoch 3/20
46/46 [==============================] - 35s 761ms/step - loss: 1.0096 - custom_f1: 0.6976 - val_loss: 1.0119 - val_custom_f1: 0.7117
Epoch 4/20
46/46 [==============================] - 32s 690ms/step - loss: 1.0040 - custom_f1: 0.6965 - val_loss: 1.0091 - val_custom_f1: 0.7117
Epoch 5/20
46/46 [==============================] - 33s 715ms/step - loss: 1.0025 - custom_f1: 0.6962 - val_loss: 1.0094 - val_custom_f1: 0.7117


In [48]:
results = model.evaluate(X_test, y_test)

200/200 [==============================] - 6s 30ms/step - loss: 1.0067 - custom_f1: 0.6957


In [49]:
print(f'F1 на тесте с дефолтным Embedding {results[1]:.2f}')

F1 на тесте с дефолтным Embedding 0.70


Попробую такие веса


navec_hudlit_v1_12B_500K_300d_100q



https://github.com/natasha/navec

In [164]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-09-04 10:43:41--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.1s    

2023-09-04 10:43:45 (16.2 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [162]:
!pip install navec

In [50]:
from navec import Navec

path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [51]:
navec['слово'].shape

(300,)

In [52]:
sorted(navec.vocab.word_ids.items(), key=lambda item: item[1], reverse=True)[:5]

[('<pad>', 500001),
 ('<unk>', 500000),
 ('ёё', 499999),
 ('ёю', 499998),
 ('ёь', 499997)]

Строим матрицу эмбедингов для предыдущего словаря, но с новыми весами

In [54]:
num_tokens = vocab_len
embedding_dim = 128
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in w2v_model.wv.key_to_index.items():
  if word in navec:
    embedding_vector = navec[word]
    embedding_matrix[i] = embedding_vector[:128]
    hits += 1
  else:
    misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 935 words (130 misses)


In [55]:
vocab_len

1065

In [57]:
initializer = tf.keras.initializers.Constant(embedding_matrix)

model = Sequential([
    Embedding(input_dim=vocab_len, output_dim=128, embeddings_initializer=initializer, input_length=300, trainable=False),
    Conv1D(128, 3),
    GlobalMaxPool1D(),
    Dense(10),
    Activation("relu"),
    Dense(num_classes),
    Activation('softmax')
])

In [58]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[custom_f1])

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 128)          136320    
                                                                 
 conv1d_2 (Conv1D)           (None, 298, 128)          49280     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 activation_5 (Activation)   (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 55        
                                                      

In [60]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
46/46 [==============================] - 24s 512ms/step - loss: 1.1098 - custom_f1: 0.6364 - val_loss: 1.0612 - val_custom_f1: 0.7117
Epoch 2/20
46/46 [==============================] - 23s 506ms/step - loss: 1.0277 - custom_f1: 0.6976 - val_loss: 1.0317 - val_custom_f1: 0.7117
Epoch 3/20
46/46 [==============================] - 23s 505ms/step - loss: 1.0144 - custom_f1: 0.6963 - val_loss: 1.0212 - val_custom_f1: 0.7117
Epoch 4/20
46/46 [==============================] - 23s 506ms/step - loss: 1.0074 - custom_f1: 0.6975 - val_loss: 1.0219 - val_custom_f1: 0.7117


In [61]:
model.evaluate(X_test, y_test)

200/200 [==============================] - 5s 23ms/step - loss: 1.0149 - custom_f1: 0.6957


[1.0148661136627197, 0.6957187056541443]

Странный способ. Без какого-либо результата

Инструкция по применению взята с https://keras.io/examples/nlp/pretrained_word_embeddings/